# Single Example Test - Citation Retrieval System

This notebook tests the full retrieval pipeline on a single example from the ScholarCopilot dataset.

**Pipeline:**
1. Load dataset and extract one citation context
2. Build retrieval resources (BM25, E5, SPECTER)
3. Run retrieval with all three methods
4. Aggregate results using RRF
5. Rerank with LLM (Hugging Face or Ollama)
6. Show final ranked papers

In [1]:
# Setup
import json
import re
import sys
from pathlib import Path
from dotenv import load_dotenv
import os

# Load environment variables
load_dotenv()

# Add parent directory to path
sys.path.insert(0, str(Path.cwd()))

print("✅ Imports successful")
print(f"📁 Working directory: {Path.cwd()}")
print(f"🔧 Inference Engine: {os.getenv('INFERENCE_ENGINE', 'ollama')}")
print(f"🤖 Local LLM: {os.getenv('LOCAL_LLM', 'gemma3:4b')}")

✅ Imports successful
📁 Working directory: /Users/ishaankalra/Dev/Retrieval/server
🔧 Inference Engine: ollama
🤖 Local LLM: gemma3:4b


## Step 1: Load Dataset and Extract Citation Context

In [3]:
import sys
from pathlib import Path

PROJECT_ROOT = Path.cwd().parent  # from .../CitationRetrieval/server to .../CitationRetrieval
sys.path.insert(0, str(PROJECT_ROOT))
from corpus_loaders.scholarcopilot import load_dataset, build_citation_corpus

# Load dataset
dataset_path = os.getenv('DATASET_DIR', 'corpus_loaders/scholarcopilot/scholar_copilot_eval_data_1k.json')
print(f"📚 Loading dataset from: {dataset_path}")

dataset = load_dataset(dataset_path)
print(f"✅ Loaded {len(dataset)} papers")

# Build corpus
print("\n🔨 Building citation corpus...")
corpus = build_citation_corpus(dataset)
print(f"✅ Corpus: {len(corpus)} documents")

📚 Loading dataset from: /Users/ishaankalra/Dev/Retrieval/corpus_loaders/scholarcopilot/scholar_copilot_eval_data_1k.json
✅ Loaded 1000 papers

🔨 Building citation corpus...
✅ Corpus: 9740 documents


In [ ]:
# ============================================================================
# CONFIGURATION: Change this to test different queries
# ============================================================================
QUERY_INDEX = 0  # Change to 1 for 2nd query, 2 for 3rd query, etc.
MAX_PAPERS_TO_SEARCH = 100  # How many papers to search through

# ============================================================================
# Extract citation contexts
# ============================================================================
cite_pattern = re.compile(r"<\|cite_\d+\|>")

print(f"🔍 Extracting citation contexts from first {MAX_PAPERS_TO_SEARCH} papers...")

all_examples = []
for paper in dataset[:MAX_PAPERS_TO_SEARCH]:
    paper_text = paper.get("paper", "")
    if not paper_text:
        continue

    bib_info = paper.get("bib_info", {})

    # Find all citation markers
    for match in cite_pattern.finditer(paper_text):
        cite_token = match.group(0)

        if cite_token not in bib_info:
            continue

        refs = bib_info[cite_token]
        if not refs:
            continue

        # Get ground truth IDs
        relevant_ids = set()
        for ref in refs:
            ref_id = ref.get("citation_key") or ref.get("paper_id")
            if ref_id:
                relevant_ids.add(str(ref_id))

        if not relevant_ids:
            continue

        # Extract context around citation (±100 words)
        pos = match.start()
        words_before = paper_text[:pos].split()[-100:]
        words_after = paper_text[match.end():].split()[:100]

        context = " ".join(words_before + words_after)
        context = re.sub(r"<\|cite_\d+\|>", "", context)
        context = " ".join(context.split())

        if len(context.split()) < 10:
            continue

        all_examples.append({
            "query": context,
            "relevant_ids": relevant_ids,
            "ground_truth_titles": [ref.get("title", "Unknown") for ref in refs],
            "paper_id": paper.get("paper_id", "unknown")
        })

print(f"✅ Found {len(all_examples)} valid citation contexts")

# Select the query at QUERY_INDEX
if QUERY_INDEX >= len(all_examples):
    raise ValueError(f"QUERY_INDEX={QUERY_INDEX} is too large. Only {len(all_examples)} queries available.")

test_example = all_examples[QUERY_INDEX]

print("\n" + "="*80)
print(f"📝 TEST EXAMPLE #{QUERY_INDEX + 1} of {len(all_examples)}")
print("="*80)
print(f"\n🔍 Query (first 300 chars):\n{test_example['query'][:300]}...")
print(f"\n✅ Ground Truth Citations ({len(test_example['relevant_ids'])}):")
for i, title in enumerate(test_example['ground_truth_titles'], 1):
    print(f"   {i}. {title}")
print(f"\n💡 To test a different query, change QUERY_INDEX (0 to {len(all_examples)-1})")
print("="*80)

## Step 2: Build Retrieval Resources

In [5]:
from src.resources.builders import build_inmemory_resources

print("🔧 Building retrieval resources...")
print("   This may take a few minutes on first run (models will be downloaded)")

resources = build_inmemory_resources(
    corpus,
    enable_bm25=True,
    enable_e5=True,
    enable_specter=True
)

print("\n✅ Resources built:")
print(f"   - BM25: {len(resources['bm25']['ids'])} documents indexed")
print(f"   - E5: {resources['e5']['corpus_embeddings'].shape[0]} embeddings")
print(f"   - SPECTER: {resources['specter']['corpus_embeddings'].shape[0]} embeddings")

🔧 Building retrieval resources...
   This may take a few minutes on first run (models will be downloaded)


/Users/ishaankalra/Dev/Retrieval/server/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Batches: 100%|██████████| 1218/1218 [09:22<00:00,  2.16batch/s]


✅ Resources built:
   - BM25: 9740 documents indexed
   - E5: 9740 embeddings
   - SPECTER: 9740 embeddings


## Step 3: Run Individual Retrievers

In [10]:
import bm25s
import torch

k = 30  # Top-k results
query = test_example['query']

# BM25
print("🔍 Running BM25...")
bm25_res = resources['bm25']
q_tokens = bm25s.tokenize(query, stopwords="en", stemmer=bm25_res["stemmer"])
doc_indices, scores = bm25_res["index"].retrieve(q_tokens, k=k)

bm25_results = []
for idx, score in zip(doc_indices[0], scores[0]):
    bm25_results.append({
        "id": bm25_res["ids"][idx],
        "title": bm25_res.get("titles", [""])[idx],
        "score": float(score),
        "source": "bm25"
    })

print(f"   ✅ Retrieved {len(bm25_results)} papers (top score: {bm25_results[0]['score']:.3f})")

# E5
print("\n🔍 Running E5...")
e5_res = resources['e5']
with torch.no_grad():
    q_emb = e5_res['model'].encode(
        [query], convert_to_tensor=True, normalize_embeddings=True, show_progress_bar=False
    )

scores = (q_emb @ e5_res['corpus_embeddings'].T)[0]
top_k = torch.topk(scores, k=min(k, len(scores)))

e5_results = []
for idx, score in zip(top_k.indices, top_k.values):
    e5_results.append({
        "id": e5_res["ids"][idx.item()],
        "title": e5_res.get("titles", [""])[idx.item()],
        "score": score.item(),
        "source": "e5"
    })

print(f"   ✅ Retrieved {len(e5_results)} papers (top score: {e5_results[0]['score']:.3f})")

# SPECTER
print("\n🔍 Running SPECTER...")
specter_res = resources['specter']
device = specter_res.get('device') or str(specter_res['corpus_embeddings'].device)

model = specter_res['model']
if str(next(model.parameters()).device) != device:
    model = model.to(device)

tokenizer = specter_res['tokenizer']

with torch.no_grad():
    inputs = tokenizer([query], padding=True, truncation=True, max_length=256, return_tensors="pt")
    inputs = {k: v.to(device) for k, v in inputs.items()}
    q_emb = model(**inputs).last_hidden_state.mean(dim=1)
    q_emb = torch.nn.functional.normalize(q_emb, dim=1)

corpus_embs = specter_res['corpus_embeddings'].to(device)
corpus_embs = torch.nn.functional.normalize(corpus_embs, dim=1)
scores = (q_emb @ corpus_embs.T)[0]
top_k = torch.topk(scores, k=min(k, len(scores)))

specter_results = []
for idx, score in zip(top_k.indices, top_k.values):
    specter_results.append({
        "id": specter_res["ids"][idx.item()],
        "title": specter_res.get("titles", [""])[idx.item()],
        "score": score.item(),
        "source": "specter"
    })

print(f"   ✅ Retrieved {len(specter_results)} papers (top score: {specter_results[0]['score']:.3f})")

🔍 Running BM25...


   ✅ Retrieved 30 papers (top score: 26.077)

🔍 Running E5...


   ✅ Retrieved 30 papers (top score: 0.897)

🔍 Running SPECTER...
   ✅ Retrieved 30 papers (top score: 0.937)


## Step 4: Aggregate with Reciprocal Rank Fusion (RRF)

In [12]:
def reciprocal_rank_fusion(results_dict, k=60):
    """Aggregate results using RRF."""
    paper_scores = {}
    
    for source, results in results_dict.items():
        for rank, paper in enumerate(results):
            paper_id = paper['id']
            rrf_score = 1.0 / (k + rank + 1)
            
            if paper_id not in paper_scores:
                paper_scores[paper_id] = {
                    'paper': paper,
                    'rrf_score': 0,
                    'sources': []
                }
            
            paper_scores[paper_id]['rrf_score'] += rrf_score
            paper_scores[paper_id]['sources'].append(source)
    
    # Sort by RRF score
    ranked = sorted(paper_scores.values(), key=lambda x: x['rrf_score'], reverse=True)
    return ranked

print("🔀 Aggregating results with RRF...")
aggregated = reciprocal_rank_fusion({
    'bm25': bm25_results,
    'e5': e5_results,
    'specter': specter_results
})

print(f"   ✅ Aggregated into {len(aggregated)} unique papers")

# Show top 5
print("\n📊 Top 5 after RRF aggregation:")
for i, item in enumerate(aggregated, 1):
    paper = item['paper']
    print(f"\n{i}. {paper['title'][:80]}...")
    print(f"   RRF Score: {item['rrf_score']:.4f}")
    print(f"   Sources: {', '.join(item['sources'])}")

🔀 Aggregating results with RRF...
   ✅ Aggregated into 74 unique papers

📊 Top 5 after RRF aggregation:

1. Lite Transformer with Long-Short Range Attention...
   RRF Score: 0.0474
   Sources: bm25, e5, specter

2. BERT: Pre-training of Deep Bidirectional Transformers for Language Understanding...
   RRF Score: 0.0401
   Sources: bm25, e5, specter

3. Tinybert: Distilling {BERT} for natural language understanding...
   RRF Score: 0.0320
   Sources: bm25, specter

4. TinyBERT: Distilling BERT for Natural Language Understanding...
   RRF Score: 0.0304
   Sources: bm25, specter

5. Funnel-transformer: Filtering out sequential redundancy for efficient language p...
   RRF Score: 0.0301
   Sources: bm25, specter

6. Distilbert, a distilled version of BERT: smaller, faster, cheaper and lighter...
   RRF Score: 0.0300
   Sources: bm25, specter

7. Q8bert: Quantized 8bit bert...
   RRF Score: 0.0296
   Sources: bm25, e5

8. A survey on visual transformer...
   RRF Score: 0.0295
   Sources: bm2

## Step 5: LLM Reranking (Hugging Face or Ollama)

In [13]:
from src.prompts.llm_reranker import LLMRerankerPrompt
from langchain_ollama import ChatOllama
from langchain_openai import ChatOpenAI
import logging

# Suppress verbose logs
logging.getLogger("httpx").setLevel(logging.WARNING)

# Prepare candidates
candidate_papers = [item['paper'] for item in aggregated[:20]]  # Top 20 for reranking

print(f"🤖 LLM Reranking {len(candidate_papers)} candidates...\n")

# Initialize LLM
inference_engine = os.getenv("INFERENCE_ENGINE", "ollama").lower()
model_id = os.getenv("LOCAL_LLM", "gemma3:4b")

if inference_engine == "ollama":
    print(f"🔄 Using Ollama with model: {model_id}")
    llm = ChatOllama(model=model_id, temperature=0)
else:
    print(f"🔄 Using Hugging Face model: {model_id}")
    from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
    from langchain_huggingface import HuggingFacePipeline
    import torch
    
    print("   Loading model (this may take a minute)...")
    tok = AutoTokenizer.from_pretrained(model_id)
    model = AutoModelForCausalLM.from_pretrained(
        model_id,
        device_map="auto",
        torch_dtype=torch.bfloat16 if torch.cuda.is_available() else torch.float32,
    )
    
    gen = pipeline(
        "text-generation",
        model=model,
        tokenizer=tok,
        max_new_tokens=1024,
        do_sample=False,
    )
    
    llm = HuggingFacePipeline(pipeline=gen)
    print("   ✅ Model loaded!")

# Build prompt
prompt = LLMRerankerPrompt(query=query, candidate_papers=candidate_papers).get_prompt()

print("\n📝 Invoking LLM...")
response = llm.invoke(prompt)
response_text = response.content

print(f"✅ LLM response received ({len(response_text)} chars)")

🤖 LLM Reranking 20 candidates...

🔄 Using Ollama with model: gemma3:4b

📝 Invoking LLM...
✅ LLM response received (645 chars)


## Step 6: Parse Results and Show Rankings

In [14]:
import json

# Parse LLM response
try:
    json_match = re.search(r"\[[\s\S]*\]", response_text)
    if json_match:
        json_str = json_match.group()
        rankings = json.loads(json_str)
    else:
        rankings = json.loads(response_text)
    
    print(f"✅ Successfully parsed {len(rankings)} rankings\n")
    
    # Build final ranked list
    final_ranked = []
    for item in rankings:
        idx = item['index'] - 1
        score = item['score']
        if 0 <= idx < len(candidate_papers):
            final_ranked.append((candidate_papers[idx], score))
    
    # Display results
    print("="*80)
    print("🏆 FINAL RANKED RESULTS (Top 10)")
    print("="*80)
    
    relevant_ids = test_example['relevant_ids']
    found_count = 0
    
    for i, (paper, score) in enumerate(final_ranked[:10], 1):
        is_correct = paper['id'] in relevant_ids
        marker = "✅ GROUND TRUTH" if is_correct else ""
        
        if is_correct:
            found_count += 1
        
        print(f"\n{i}. {paper['title'][:100]}...")
        print(f"   LLM Score: {score:.3f} {marker}")
        print(f"   ID: {paper['id']}")
    
    print("\n" + "="*80)
    print(f"📊 EVALUATION")
    print("="*80)
    print(f"Ground truth citations found in top 10: {found_count}/{len(relevant_ids)}")
    print(f"Recall@10: {found_count/len(relevant_ids):.2%}")
    
except Exception as e:
    print(f"❌ Error parsing LLM response: {e}")
    print(f"\nRaw response (first 500 chars):\n{response_text[:500]}")

✅ Successfully parsed 20 rankings

🏆 FINAL RANKED RESULTS (Top 10)

1. BERT: Pre-training of Deep Bidirectional Transformers for Language Understanding...
   LLM Score: 0.980 
   ID: devlin2019bert

2. Lite Transformer with Long-Short Range Attention...
   LLM Score: 0.950 
   ID: lite_transformer

3. Tinybert: Distilling {BERT} for natural language understanding...
   LLM Score: 0.850 
   ID: jiao2019tinybert

4. TinyBERT: Distilling BERT for Natural Language Understanding...
   LLM Score: 0.800 
   ID: jiao20tinybert

5. Funnel-transformer: Filtering out sequential redundancy for efficient language processing...
   LLM Score: 0.750 
   ID: funnel

6. Distilbert, a distilled version of BERT: smaller, faster, cheaper and lighter...
   LLM Score: 0.700 
   ID: distillbert

7. Q8bert: Quantized 8bit bert...
   LLM Score: 0.650 
   ID: zafrir2019q8bert

8. A survey on visual transformer...
   LLM Score: 0.600 
   ID: han2020survey

9. Compressing large-scale transformer-based models: A ca

## Summary

This notebook demonstrated the complete citation retrieval pipeline:
1. ✅ Loaded citation context from ScholarCopilot dataset
2. ✅ Built BM25, E5, and SPECTER retrieval indices
3. ✅ Retrieved top-k candidates from each method
4. ✅ Aggregated using Reciprocal Rank Fusion
5. ✅ Reranked with LLM (Hugging Face or Ollama)
6. ✅ Evaluated against ground truth

**To run full evaluation:**
```bash
python compare_baselines_vs_system.py --num-examples 500 --use-dspy --llm-reranker --output-dir final --k 20
```